# Artículo para Science of the Total Environment

* Nuevos datos base
* Ajustar PCR
* Ajustar PLS
* Métricas de cross-validation

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
import time

from covid19_vulnerabilidad_mex.datos import *
from covid19_vulnerabilidad_mex.vulnerabilidad import *

import pandas as pd
import geopandas as gpd

import glob
import os
import matplotlib.pyplot as plt
import numpy as np

from datetime import timedelta
import datetime
from datetime import timedelta, date, datetime
import pickle

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.cross_decomposition import PLSRegression

import seaborn as sns
import imageio
from pygifsicle import optimize

plt.style.use('ggplot')

In [ ]:
%load_ext autoreload
%autoreload 2

## Nuevos datos base

La idea es probar los índices con diferentes selecciones de variables, para eso vamos a empezar con las variables que usamos para el COVID-SVI, que son realmente muy pocas y después ver si PCR o PLS son capaces de obtener más información a partir de ir agregando nuevas variables.

Lo primero es entonces hacer nuestra nueva base de datos de variables. Empezamos a partir de `variables_reclasificadas.csv` y usamos `dicionario_variables_svi_covid.csv` para seleccionar las variables base a utilizar. 

In [ ]:
diccionario = pd.read_csv("datos/diccionario_variables_svi_covid.csv").rename({'COVID-SVI ':'COVID-SVI'}, axis=1)
diccionario.head()

,campo,descriptor,COVID-SVI,BASE-SVI,display_name
0,pt_2015,"población total,2015",Demographic,NaN,Population
1,pob_menore,Personas de 0 a 5 años de edad.,Demographic,Household Composition,Population under 5 years
2,pob_mayores,Personas de más de 65,Demographic,Household Composition,Population over 65 years
3,pct_hipertension,Porcentaje de personas con hipertensión,Epidemiological,NaN,Prevalence of hypertension
4,pct_diabetes,Porcentaje de personas con diabetes,Epidemiological,NaN,Prevalence of diabetes


La columna `COVID-SVI` nos dice qué variables vamos a utilizar y las dimensiones a las que pertenecen

In [ ]:
diccionario_covid_svi = diccionario[diccionario['COVID-SVI'].notnull()]
diccionario_covid_svi

,campo,descriptor,COVID-SVI,BASE-SVI,display_name
0,pt_2015,"población total,2015",Demographic,NaN,Population
1,pob_menore,Personas de 0 a 5 años de edad.,Demographic,Household Composition,Population under 5 years
2,pob_mayores,Personas de más de 65,Demographic,Household Composition,Population over 65 years
3,pct_hipertension,Porcentaje de personas con hipertensión,Epidemiological,NaN,Prevalence of hypertension
4,pct_diabetes,Porcentaje de personas con diabetes,Epidemiological,NaN,Prevalence of diabetes
5,pct_obesidad,Porcentaje de personas con obesidad,Epidemiological,NaN,Prevalence of obesity
6,ic_asalud,%Carencia por acceso a los servicios de salud,Health Care,NaN,Lack in health care access (%)
7,ic_segsoc,%Carencia por acceso a la seguridad social,Health Care,NaN,Lack in social security (%)
8,total_camas_publicos,Unidades económicas dedicadas principalmente a...,Health Care,NaN,Public hospital beds
9,total_camas_privados,Unidades económicas dedicadas principalmente a...,Health Care,NaN,Private hospital beds


In [ ]:
id_vars = ['cvegeo', 'nom_ent']
vars_svi_covid = diccionario_covid_svi.campo.values
indicadores = pd.read_csv("datos/municipios/indicadores_reclasificados.csv", dtype={'cvegeo':str})
indicadores_svi_covid = indicadores[id_vars + list(vars_svi_covid)]
indicadores_svi_covid.head()

KeyError: "['pob_mayores'] not in index"

In [ ]:
id_vars + list(vars_svi_covid)

['cvegeo',
 'nom_ent',
 'pt_2015',
 'pob_menore',
 'pob_mayores',
 'pct_hipertension',
 'pct_diabetes',
 'pct_obesidad',
 'ic_asalud',
 'ic_segsoc',
 'total_camas_publicos',
 'total_camas_privados',
 'total_hospitales_publicos',
 'vna_2015',
 'vnd_2015',
 'vne_2015',
 'ic_cv',
 'ic_sbv',
 'an_2015',
 'lmex_2015',
 'pobreza',
 'vul_ing',
 'npnv',
 'ic_rezedu',
 'carencias',
 'plbm']

In [ ]:
pd.read_csv("datos/municipios/indicadores_reclasificados.csv", dtype={'cvegeo':str})

,cvegeo,nom_ent,pt_2015,an_2015,ne614_015,bi_2015,sins_15,vpt_2015,vns_2015,vna_2015,...,entidad_cvegeo,pob01,pob2020,pct_obesidad,pct_hipertension,pct_diabetes,total_camas_publicos,total_camas_privados,total_hospitales_publicos,total_hospitales_privados
0,01001,Aguascalientes,877190,2.0570,3.4786,25.6873,14.1560,0.5478,0.5543,0.6627,...,1,797010,961977,31.486541,14.942242,7.495861,0.959461,0.993091,0.981481,0.965385
1,01002,Aguascalientes,46464,4.4207,2.5959,41.8921,5.5355,1.6779,5.6457,1.2979,...,1,45492,50864,32.282284,15.320425,7.953634,0.959461,0.993091,0.981481,0.965385
2,01003,Aguascalientes,56048,4.7805,4.3906,49.1856,9.7524,1.0577,0.7756,0.9519,...,1,54136,60760,40.004293,13.751906,9.172624,0.959461,0.993091,0.981481,0.965385
3,01004,Aguascalientes,15577,4.3233,2.6265,33.0597,5.1165,1.5629,2.8986,0.5968,...,1,15042,16918,32.596450,16.431493,7.383116,0.959461,0.993091,0.981481,0.965385
4,01005,Aguascalientes,120405,3.2445,4.1297,33.7271,13.8341,0.9201,1.0273,0.9443,...,1,99590,130184,34.731715,12.356755,6.745819,0.959461,0.993091,0.981481,0.965385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,32054,Zacatecas,19155,8.1332,3.6565,42.1699,8.8384,2.5438,8.9568,3.8478,...,32,18490,20177,31.075955,16.920279,9.895491,0.991437,1.000000,0.824074,0.909615
2453,32055,Zacatecas,30240,4.7985,3.4198,53.7212,13.5516,0.9201,4.8484,1.7459,...,32,29395,31804,30.526273,21.746589,12.050064,0.991437,1.000000,0.824074,0.909615
2454,32056,Zacatecas,146147,1.2432,2.1746,23.0304,14.0968,0.2485,1.2004,1.6826,...,32,138176,155533,36.822130,20.010453,11.814172,0.991437,1.000000,0.824074,0.909615
2455,32057,Zacatecas,19413,5.6122,2.5997,42.0488,9.0609,1.1950,6.9448,3.0891,...,32,16934,20285,32.477646,15.941678,8.089966,0.991437,1.000000,0.824074,0.909615
